https://www.youtube.com/watch?v=cAGliEJV9_o&t=10s  

Decorators give us a slightly different way to manipulate clases at definition. It does NOT require inheritance (like `__init_subclass__` and `meta`)

1.Modify the class at definition?
- `__init_subclass__` if this behavior should propagate to all subclasses?
- class decorator to avoid inheritance ‘side effects’
2. Modify the class at instantiation?
- Inheritance > decorator that returns class wrapped

## Decorating using a Function

In [2]:
# write a class decorator the same way as a function decorator
def add_some_method(cls):
    def foo(self, base=0):
        print(f'Called from {self.__class__.__name__}')
        return base + 5

    cls.foo = foo
    return cls

@add_some_method
class A:
    pass

a = A()
a.foo(10)

Called from A


15

In the above, we rely on the decorator to dynamically add a method to the class on definition. The function within is NOT a wrapper

## Using a Class as a Decorator?

In [4]:
# from http://www.dontusethiscode.com/blog/py-decorators-adv.html
class Registry:
    all_funcs = []

    def __new__(cls, func):
        cls.all_funcs.append(func)

        # pass the function through instead
        #   of returning an instance of this class
        return func

    @classmethod
    def execute_registered(cls):
        for f in cls.all_funcs:
            print(f'{f.__name__}() = {f()}')

@Registry
def f1():
    return 1

@Registry
def f2():
    return 2

@Registry
def f3():
    return 3

print(f1, f2, f3, sep='\n')
Registry.execute_registered()

<function f1 at 0x7fd86d7d20d0>
<function f2 at 0x7fd86d7d2280>
<function f3 at 0x7fd86d7d2310>
f1() = 1
f2() = 2
f3() = 3


## Ambiguously Parameterized Decorators 

As made convenient by decorator.decorator, and in the source code for @dataclass there is a way to implement a decorator such that either take parameterized arguments, or no arguments and ‘just work’. I use this term loosely because I personally don’t like the ambiguity introduced by this design pattern. Nonetheless, in the spirit of Python: we’re all consenting adults.

In [5]:
def maybe_dec_factory(f=None, /, *, kwarg1=None, kwarg2=None):
    def dec(f):
        print(f'decorating {f.__name__} with {kwarg1 = }, {kwarg2 = }')

        def wrapper(*args, **kwargs):
            print(f'before {f}')
            result = f(*args, **kwargs)
            print(f'after {f}')
            return result
        return wrapper

    if f is None:
        return dec
    return dec(f)

@maybe_dec_factory
def f1():
    print('executing f1')

@maybe_dec_factory(kwarg1='hello', kwarg2='world')
def f2():
    print('executing f2')

decorating f1 with kwarg1 = None, kwarg2 = None
decorating f2 with kwarg1 = 'hello', kwarg2 = 'world'


## Example using Decorated Class

In [6]:
from functools import partial, wraps

def debug(prefix=''): # this outer function provides an "environment" for the inner functions
    def decorate(func):
        msg = prefix + func.__qualname__

        @wraps(func)
        def wrapper(*args, **kwargs):
            print(msg)
            return func(*args, **kwargs)

        return wrapper

    return decorate


def debugmethods(cls):
    for key, val in vars(cls).items():
        if callable(val):
            setattr(cls, key, debug(val))

    return cls

"brain surgery" with class decorators, debugging attribute access

In [7]:
def debugattr(cls):
    orig_getattribute = cls.__getattribute__

    def __getattribute__(self, name):
        print(f'Get: {name}')
        return orig_getattribute(self, name)

    cls.__getattribute__ = __getattribute__

    return cls

In [11]:
@debugattr
class A:
    def __init__(self, a) -> None:
        self.a = a

In [12]:
a = A(1)

In [13]:
a.a

Get: a


1

In [ ]:
class LoggingMeta